In [72]:
from nbdev import *

In [73]:
%nbdev_hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create JNK artifact from JNK data "before handling missing"

In [74]:
import pandas as pd
import numpy as np
from fastcore.all import *
import wandb
from datetime import datetime, timedelta
from timecluster_extension.load import *

In [75]:
base_path_JNK = Path.home()/'data/PACMEL-2019/JNK/' # *
[path for path in base_path_JNK.ls()]

[Path('/home/user/data/PACMEL-2019/JNK/df_bool_dict.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/df_categorical_dict.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/df_jnk_base.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_before_handling_missing.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_clusters_nk.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_clusters_npk.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_tslearn_cutt_0.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_tslearn_cutt_mean.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_tslearn_mul_0.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolat

In [76]:
jnk_files = []
filepaths_pickle = base_path_JNK.ls(file_exts='.pickle')
for i, path in enumerate(filepaths_pickle):
    f = open(path, 'rb')    
    bin_data = f.read()
    print(f'Loading file {i}...')
    jnk_files.append(pickle.loads(bin_data))

Loading file 0...
Loading file 1...
Loading file 2...
Loading file 3...
Loading file 4...
Loading file 5...
Loading file 6...
Loading file 7...
Loading file 8...
Loading file 9...
Loading file 10...
Loading file 11...
Loading file 12...
Loading file 13...


In [77]:
df = jnk_files[3][0]
df.index.freq = 's'

In [78]:
df = df.interpolate(limit_direction='both')

In [79]:
# Note: normalization is now done when creating the artifact, to store the means and stds in the metadata
#df = normalize_columns(df)

In [80]:
df = df.resample('5s').mean()

In [81]:
#17280 muestras = 1 dia
# 10 dias = 172800
# training df[0:172800]         Normalize
# validation df[172800:224640]  Don't normalize
# testing df[224640:276481]     Don't normalize

In [82]:
df = df[0:172800] 

In [83]:
ruta_wandb_artifacts = 'data/PACMEL-2019/wandb_artifacts'# When permissions are fixed, save artifacts in 'data/PACMEL-2019/wandb_artifacts'
ar = TSArtifact.from_df(df, 'JNK', normalize=True, path= Path.home()/ruta_wandb_artifacts)

In [84]:
ar.metadata

{'TS': {'sd': '2019-06-01 00:00:00',
  'ed': '2019-06-10 23:59:55',
  'created': 'from-df',
  'freq': '<5 * Seconds>',
  'n_vars': 16,
  'n_samples': 172800,
  'has_missing_values': 'False',
  'vars': ['RCD_AverageThree-phaseCurrent',
   'LCD_AverageThree-phaseCurrent',
   'LP_AverageThree-phaseCurrent',
   'LHD_LeftHaulageDrive(tractor)Temperature(gearbox)',
   'RHD_RightHaulageDrive(tractor)Temperature(gearbox)',
   'LA_LeftArmTemperature',
   'RA_RightArmTemperature',
   'SM_DailyRouteOfTheShearer',
   'SM_TotalRoute',
   'LHD_EngineCurrent',
   'RHD_EngineCurrent',
   'RCD_BearingTemperature',
   'SM_ShearerSpeed',
   'SM_ShearerLocation',
   'SM_ShearerMoveInLeft',
   'SM_ShearerMoveInRight'],
  'normalization': {'means': {'RCD_AverageThree-phaseCurrent': 37.00287326388889,
    'LCD_AverageThree-phaseCurrent': 35.23550983796297,
    'LP_AverageThree-phaseCurrent': 1.6554114583333333,
    'LHD_LeftHaulageDrive(tractor)Temperature(gearbox)': 55.24939930555555,
    'RHD_RightHaulageD

In [85]:
run = wandb.init(entity = 'pacmel', project='timecluster-extension', job_type='create_dataset', resume=True)
run.log_artifact(ar,aliases=['train_10days'])

wandb: Wandb version 0.10.8 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Streaming file created twice in same run: /home/user/work/nbs/wandb/run-20201027_173944-3dwqdbrr/wandb-events.jsonl
